<a href="https://colab.research.google.com/github/DaiVo20/za_traffic_detection/blob/main/keras_retinanet/data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import cv2
import random
import numpy as np
import pandas as pd


def colorjitter(img, cj_type="b"):
    '''
    ### Different Color Jitter ###
    img: image
    cj_type: {b: brightness, s: saturation, c: constast}
    '''
    if cj_type == "b":
        # value = random.randint(-50, 50)
        value = np.random.choice(np.array([-50, -40, -30, 30, 40, 50]))
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(hsv)
        if value >= 0:
            lim = 255 - value
            v[v > lim] = 255
            v[v <= lim] += value
        else:
            lim = np.absolute(value)
            v[v < lim] = 0
            v[v >= lim] -= np.absolute(value)

        final_hsv = cv2.merge((h, s, v))
        img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
        return img
    
    elif cj_type == "s":
        # value = random.randint(-50, 50)
        value = np.random.choice(np.array([-50, -40, -30, 30, 40, 50]))
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(hsv)
        if value >= 0:
            lim = 255 - value
            s[s > lim] = 255
            s[s <= lim] += value
        else:
            lim = np.absolute(value)
            s[s < lim] = 0
            s[s >= lim] -= np.absolute(value)

        final_hsv = cv2.merge((h, s, v))
        img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
        return img
    
    elif cj_type == "c":
        brightness = 10
        contrast = random.randint(40, 100)
        dummy = np.int16(img)
        dummy = dummy * (contrast/127+1) - contrast + brightness
        dummy = np.clip(dummy, 0, 255)
        img = np.uint8(dummy)
        return img


def filters(img, f_type = "blur"):
    '''
    ### Filtering ###
    img: image
    f_type: {blur: blur, gaussian: gaussian, median: median}
    '''
    if f_type == "blur":
        image=img.copy()
        fsize = 3
        return cv2.blur(image,(fsize,fsize))
    
    elif f_type == "gaussian":
        image=img.copy()
        fsize = 3
        return cv2.GaussianBlur(image, (fsize, fsize), 0)
    
    elif f_type == "median":
        image=img.copy()
        fsize = 3
        return cv2.medianBlur(image, fsize)

In [ ]:
idx2sign = {
    1: "No entry", # Cấm ngược chiều
    2: "No parking / waiting", # Cấm dừng và đỗ
    3: "No turning", # Cấm rẽ
    4: "Max Speed", # Giới hạn tốc độ
    5: "Other prohibition signs", # Cấm còn lại
    6: "Warning", # Nguy hiểm
    7: "Mandatory" # Hiệu lệnh
}

In [ ]:
df_train_croped = pd.read_csv('/content/drive/MyDrive/TrafficSign/Model/RetinaNet/data_crop/image_split_4/traffic_train/train_croped.csv')
df_train_croped['class'] = df_train_croped['label'].replace(idx2sign)
df_train_croped.drop(columns=['width', 'height', 'image_split', 'label'], axis=1, inplace=True)
df_train_croped

,file_name,xmin,ymin,xmax,ymax,class
0,/content/drive/MyDrive/TrafficSign/Model/Retin...,70,19,89,37,No parking / waiting
1,/content/drive/MyDrive/TrafficSign/Model/Retin...,259,41,342,124,No turning
2,/content/drive/MyDrive/TrafficSign/Model/Retin...,376,23,393,38,Warning
3,/content/drive/MyDrive/TrafficSign/Model/Retin...,214,2,289,74,No parking / waiting
4,/content/drive/MyDrive/TrafficSign/Model/Retin...,290,0,328,22,Other prohibition signs
...,...,...,...,...,...,...
5565,/content/drive/MyDrive/TrafficSign/Model/Retin...,147,69,194,116,Mandatory
5566,/content/drive/MyDrive/TrafficSign/Model/Retin...,297,118,317,137,No turning
5567,/content/drive/MyDrive/TrafficSign/Model/Retin...,92,0,365,76,Mandatory
5568,/content/drive/MyDrive/TrafficSign/Model/Retin...,123,15,171,56,No parking / waiting


In [ ]:
dir_path = '/content/drive/MyDrive/TrafficSign/Model/RetinaNet/data_crop/data_augmentation/images'
# dir_path = '/content/data_augmentation/images'

if not os.path.exists(dir_path):
    os.makedirs(dir_path)

In [ ]:
train_images = df_train_croped['file_name'].unique()

total = len(train_images)
for idx, img_filename in enumerate(train_images):
    print(f'{idx}/{total}')
    image = cv2.imread(img_filename)
    img_name = img_filename.split('/')[-1].split('.')[0]

    # Brightness
    b_img = colorjitter(image, cj_type="b")
    cv2.imwrite(f"{dir_path}/{img_name}_brightness.png", b_img)

    # Saturation
    s_img = colorjitter(image, cj_type="s")
    cv2.imwrite(f"{dir_path}/{img_name}_saturation.png", s_img)

    # Contrast
    c_img = colorjitter(image, cj_type="c")
    cv2.imwrite(f"{dir_path}/{img_name}_contrast.png", c_img)

    # Blur
    blur_img = filters(image, f_type = "blur")
    cv2.imwrite(f"{dir_path}/{img_name}_blur.png", blur_img)

    # Original image
    cv2.imwrite(f"{dir_path}/{img_name}.png", image)

In [ ]:
df_train_croped

,file_name,xmin,ymin,xmax,ymax,class
0,/content/drive/MyDrive/TrafficSign/Model/Retin...,70,19,89,37,No parking / waiting
1,/content/drive/MyDrive/TrafficSign/Model/Retin...,259,41,342,124,No turning
2,/content/drive/MyDrive/TrafficSign/Model/Retin...,376,23,393,38,Warning
3,/content/drive/MyDrive/TrafficSign/Model/Retin...,214,2,289,74,No parking / waiting
4,/content/drive/MyDrive/TrafficSign/Model/Retin...,290,0,328,22,Other prohibition signs
...,...,...,...,...,...,...
5565,/content/drive/MyDrive/TrafficSign/Model/Retin...,147,69,194,116,Mandatory
5566,/content/drive/MyDrive/TrafficSign/Model/Retin...,297,118,317,137,No turning
5567,/content/drive/MyDrive/TrafficSign/Model/Retin...,92,0,365,76,Mandatory
5568,/content/drive/MyDrive/TrafficSign/Model/Retin...,123,15,171,56,No parking / waiting


In [ ]:
df_original = df_train_croped.copy()
df_brightness = df_train_croped.copy()
df_saturation = df_train_croped.copy()
df_contrast = df_train_croped.copy()
df_blur = df_train_croped.copy()

In [ ]:
path_dir_image = '/content/drive/MyDrive/TrafficSign/Model/RetinaNet/data_crop/data_augmentation/images/'

In [ ]:
def replace_path(file_name):
    img_name = file_name.split('/')[-1].split('.')[0]
    return path_dir_image + img_name + '.png'

df_original['file_name'] = df_original['file_name'].apply(replace_path)

In [ ]:
def replace_path(file_name):
    img_name = file_name.split('/')[-1].split('.')[0]
    return path_dir_image + img_name + '_brightness.png'

df_brightness['file_name'] = df_brightness['file_name'].apply(replace_path)

In [ ]:
def replace_path(file_name):
    img_name = file_name.split('/')[-1].split('.')[0]
    return path_dir_image + img_name + '_saturation.png'

df_saturation['file_name'] = df_saturation['file_name'].apply(replace_path)

In [ ]:
def replace_path(file_name):
    img_name = file_name.split('/')[-1].split('.')[0]
    return path_dir_image + img_name + '_contrast.png'

df_contrast['file_name'] = df_contrast['file_name'].apply(replace_path)

In [ ]:
def replace_path(file_name):
    img_name = file_name.split('/')[-1].split('.')[0]
    return path_dir_image + img_name + '_blur.png'

df_blur['file_name'] = df_blur['file_name'].apply(replace_path)

In [ ]:
df_augment = pd.concat([df_original, df_brightness, df_saturation, df_contrast, df_blur], ignore_index=True)
df_augment.to_csv('/content/drive/MyDrive/TrafficSign/Model/RetinaNet/data_crop/data_augmentation/train_augmentation.csv', index=False, header=False)

In [3]:
df_augment = pd.read_csv('/content/drive/MyDrive/TrafficSign/Model/RetinaNet/data_crop/data_augmentation/train_augmentation.csv', header=None)

In [4]:
df_augment

,0,1,2,3,4,5
0,/content/drive/MyDrive/TrafficSign/Model/Retin...,70,19,89,37,No parking / waiting
1,/content/drive/MyDrive/TrafficSign/Model/Retin...,259,41,342,124,No turning
2,/content/drive/MyDrive/TrafficSign/Model/Retin...,376,23,393,38,Warning
3,/content/drive/MyDrive/TrafficSign/Model/Retin...,214,2,289,74,No parking / waiting
4,/content/drive/MyDrive/TrafficSign/Model/Retin...,290,0,328,22,Other prohibition signs
...,...,...,...,...,...,...
27845,/content/drive/MyDrive/TrafficSign/Model/Retin...,147,69,194,116,Mandatory
27846,/content/drive/MyDrive/TrafficSign/Model/Retin...,297,118,317,137,No turning
27847,/content/drive/MyDrive/TrafficSign/Model/Retin...,92,0,365,76,Mandatory
27848,/content/drive/MyDrive/TrafficSign/Model/Retin...,123,15,171,56,No parking / waiting


In [6]:
df_augment.shape[0]

27850

In [8]:
len(df_augment[0].unique())

18490